In [8]:
!pip install newspaper3k

  Stored in directory: C:\Users\shrikanth\AppData\Local\pip\Cache\wheels\de\03\ca\778e3a7a627e3d98836cc890e7cb40c7575424cfd3340f40ed
  Stored in directory: C:\Users\shrikanth\AppData\Local\pip\Cache\wheels\81\2b\43\a02ede72324dd40cdd7ca53aad718c7710628e91b8b0dc0f02
  Stored in directory: C:\Users\shrikanth\AppData\Local\pip\Cache\wheels\83\15\9c\a3f1f67e7f7181170ad37d32e503c35da20627c013f438ed34
  Stored in directory: C:\Users\shrikanth\AppData\Local\pip\Cache\wheels\8c\69\b7\f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
Successfully built feedfinder2 tinysegmenter jieba3k feedparser


In [10]:
!pip install scrapy

  Stored in directory: C:\Users\shrikanth\AppData\Local\pip\Cache\wheels\88\99\96\cfef6665f9cb1522ee6757ae5955feedf2fe25f1737f91fa7f
  Stored in directory: C:\Users\shrikanth\AppData\Local\pip\Cache\wheels\51\01\d1\4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
Successfully built PyDispatcher protego
  Found existing installation: attrs 19.1.0
    Uninstalling attrs-19.1.0:
      Successfully uninstalled attrs-19.1.0


In [22]:
!pip install requests

In [34]:
!pip install beautifulsoup4

In [10]:
!pip install validator-collection

In [1]:
import datetime as dt
import pandas as pd
import numpy as np

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from validator_collection import validators, checkers
from urllib.request import urlopen
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep
from newspaper import Article
import scrapy
import requests
import time

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shrikanth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
def get_news_class_hrefs(url):
    """
    Finds all urls pointed to by all links inside
    'news' class div elements
    """
    #https://stackoverflow.com/questions/36709165/beautifulsoup-object-of-type-response-has-no-len
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    #https://stackoverflow.com/questions/32468801/how-to-get-href-from-an-a-tag-inside-a-div
    links = ['https://timesofindia.indiatimes.com'+a['href'] for div in soup.find_all("div", attrs={"class": "content"}) for a in div.find_all('a')]
    return links[1:]

In [50]:
get_news_class_hrefs('https://timesofindia.indiatimes.com/topic/Politics/2')

['https://timesofindia.indiatimes.com/india/bjps-virtual-political-campaign/articleshow/76025044.cms',
 'https://timesofindia.indiatimes.comhttps://timesofindia.indiatimes.com/a-glimpse-of-amitabh-bachchans-illustrious-life/political-plunge/photostory/75025659.cms',
 'https://timesofindia.indiatimes.com/city/bengaluru/places-of-worship-gear-up-to-welcome-devotees-in-karnataka/articleshow/76241539.cms',
 'https://timesofindia.indiatimes.com/city/bengaluru/karnataka-hunt-on-to-find-bs-yediyurappas-successor/articleshow/76241457.cms',
 'https://timesofindia.indiatimes.com/city/rajkot/politics-sullies-corona-warriors-felicitation/articleshow/76329286.cms',
 'https://timesofindia.indiatimes.com/city/hyderabad/cong-cheap-politics-by-kcr/articleshow/75636801.cms',
 'https://timesofindia.indiatimes.comhttps://timesofindia.indiatimes.com/life-style/spotlight/most-hilarious-whatsapp-messages-about-coronavirus-because-laughter-is-the-best-medicine/the-political-game/photostory/74654509.cms',
 'ht

In [19]:
def get_news_details(url):
    
    try:
        toi_article = Article(url, language="en")
        toi_article.download()
        time.sleep(0.5)
        toi_article.parse()
        time.sleep(2)
        toi_article.nlp()
    
        ArticleTitle = toi_article.title
        ArticleText = toi_article.text
        ArticleSummary = toi_article.summary
        #ArtPubDate = toi_article.publish_date
        return ArticleTitle, ArticleText, ArticleSummary

    except ArticleException:
        toi_article = Article(url, language="en")
        toi_article.download()
        time.sleep(2)
        toi_article.parse()
        time.sleep(2)
        toi_article.nlp()
    
        ArticleTitle = toi_article.title
        ArticleText = toi_article.text
        ArticleSummary = toi_article.summary
        #ArtPubDate = toi_article.publish_date
        return ArticleTitle, ArticleText, ArticleSummary
    
    else:
        pass
        
    

In [29]:
TOI_Articles = pd.DataFrame(columns = ['Title', 'Text', 'Summary'])
invalid_url, ArtTitle, ArtTest, ArtSummary, processedURLs = ([] for i in range(5))
for i in range(20,500):
    #https://timesofindia.indiatimes.com/topic/Politics
    pageUrl = 'https://timesofindia.indiatimes.com/topic/Politics'+'/{}'.format(i)
    processedURLs.append(pageUrl)
    ArtLinks = get_news_class_hrefs(pageUrl)
    
    #https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests
    for artLink in ArtLinks:
        if checkers.is_url(artLink):
            Title, Text, Summary = get_news_details(artLink)
            temp_df = pd.DataFrame({'Title' : Title,
                                    'Text' : Text,
                                    'Summary' : Summary}, index=[0])
            TOI_Articles = TOI_Articles.append(temp_df, ignore_index=True, sort=True)
            ArtTitle.append(Title)
            ArtTest.append(Text)
            ArtSummary.append(Summary)
        else:
            invalid_url.append(artLink)
    print('Total News Articles Scrapped =', len(ArtTitle))
    


Total News Articles Scrapped = 17
Total News Articles Scrapped = 33
Total News Articles Scrapped = 49
Total News Articles Scrapped = 65
Total News Articles Scrapped = 81
Total News Articles Scrapped = 97
Total News Articles Scrapped = 113
Total News Articles Scrapped = 129
Total News Articles Scrapped = 145
Total News Articles Scrapped = 161
Total News Articles Scrapped = 177
Total News Articles Scrapped = 193
Total News Articles Scrapped = 209
Total News Articles Scrapped = 225
Total News Articles Scrapped = 241
Total News Articles Scrapped = 257
Total News Articles Scrapped = 273
Total News Articles Scrapped = 289
Total News Articles Scrapped = 305
Total News Articles Scrapped = 321
Total News Articles Scrapped = 337
Total News Articles Scrapped = 353
Total News Articles Scrapped = 369
Total News Articles Scrapped = 385
Total News Articles Scrapped = 401
Total News Articles Scrapped = 417
Total News Articles Scrapped = 433
Total News Articles Scrapped = 449
Total News Articles Scrapp

NameError: name 'ArticleException' is not defined

In [31]:
TOI_Articles.to_csv(r'C:\Users\shrikanth\Desktop\University of Passau\Text Mining Project\Project Implementation\CSV Files\TOI\TOI_II.csv', 
                    index=False, header=True)

In [30]:
TOI_Articles.tail()

,Summary,Text,Title
1543,NEW DELHI: Senior BJP leader and Union ministe...,NEW DELHI: Senior BJP leader and Union ministe...,Congress is creating 'political pollution': Naqvi
1544,Read the full articleJoin us and get access to...,Read the full article\n\nJoin us and get acces...,How politics is changing office politics
1545,Lucknow: The Congress party on Sunday said tha...,Lucknow: The Congress party on Sunday said tha...,‘Cases against Lallu politically motivated’
1546,Spats over ideology are rising in the workplac...,Spats over ideology are rising in the workplac...,How politics is changing office politics
1547,Manipur assembly speaker Yumnam Khemchand Sing...,Manipur assembly speaker Yumnam Khemchand Sing...,Yumnam Khemchand Singh: Opposition submits not...


In [15]:
text_format = {'Title' : ArtTitle, 'Text' : ArtTest, 'Summary' : ArtSummary}
my_df = pd.DataFrame(text_format)
my_df.to_csv(r'C:\Users\shrikanth\Desktop\University of Passau\Text Mining Project\Project Implementation\CSV Files\TOI\TOI_II.csv', 
            index=False, header=True)